# Working with Request Body Parameters

One of the more challenging aspects of using an API is know what data is required, and whether or not you are proving *valid* data.  The halutz client will use the Swagger spec to validate your requests.  In truth, the halutz library is using the [python-jsonschema-objects](https://github.com/cwacek/python-jsonschema-objects) library for this function.  The halutz client adds some helpers as well.  Let's begin with first creating a client instance:

In [2]:
# %load docs_client.py
from halutz.client import Client
import json

server_url = 'http://localhost:32768'
swagger_spec = json.load(open('swagger_spec.json'))

client = Client(server_url, origin_spec=swagger_spec)


## Requests with Body Parameters

Not all Requests have body parameters.  You can easily tell by checking the Request `body_param` attribute.  For example, this request does not have a body parameter:

In [3]:
rqst = client.request.ipam.ipam_vlans_list
print rqst.body_param

None


Now let's use a Request that does have a body parameter

In [4]:
rqst = client.request.ipam.ipam_vlans_create
print rqst.body_param

data


Here we can see the request has a body parameter called `data`.  We can introspect the request as well:

In [5]:
rqst

Request: {
   "path": "/api/ipam/vlans/", 
   "params": [
      "data"
   ], 
   "method": "post"
}

In [6]:
rqst.params[rqst.body_param].param_spec

{u'in': u'body',
 u'name': u'data',
 u'schema': {u'properties': {u'custom_fields': {u'description': u'',
    u'type': u'string'},
   u'description': {u'description': u'', u'type': u'string'},
   u'group': {u'description': u'', u'type': u'string'},
   u'name': {u'description': u'', u'type': u'string'},
   u'role': {u'description': u'', u'type': u'string'},
   u'site': {u'description': u'', u'type': u'string'},
   u'status': {u'description': u'', u'type': u'string'},
   u'tenant': {u'description': u'', u'type': u'string'},
   u'vid': {u'description': u'', u'type': u'integer'}},
  u'required': [u'vid', u'name'],
  u'type': u'object'}}

## Accessing Body Fields

When the halutz library provides you the Request instance it will automically add the body parameter as an attribute.  In the above example, the request instance would have another attribute called `data`.  

<div class="alert alert-warning">
<strong>NOTICE! &nbsp;</strong> This body instance variable is dynamically created based on the Swagger spec.  When you introspect the variable, you may or may not see a humaized name for the resulting class.  If th Swagger spec has an 'x-model' tag assigned then you will see that name used; otherwise you will see a numberic value, as in the case below.
</div>

In [7]:
rqst.data

<4441882704 custom_fields=None description=None group=None name=None role=None site=None status=None tenant=None vid=None>

Alternatively, you can get a listing of the request body fields using the `keys()` method:

In [8]:
rqst.data.keys()

[u'status',
 u'group',
 u'name',
 u'vid',
 u'site',
 u'role',
 u'custom_fields',
 u'tenant',
 u'description']

You can then assign values to these fields using the attribute.  For example, let's assign the `vid` and `name` fields.

In [9]:
rqst.data.vid = 10
rqst.data.name = 'Purple'

## Validating Body Data

As a function of using the `python-jsonschema-objects` library, any time you assign a value the value is checked against the jsonschema to determine whether or not it's valid.  

For example, if you try to assign an integer to a string field, you will get a `ValidationError` exception.  Here is such an example:

In [87]:
try:
   rqst.data.vid = "foobaz"     # vid should be an int, so this will raise an Exception
except Exception as exc:
    print ">>> Validation error: '%s'" % exc

>>> Validation error: 'foobaz is not an integer'


You can also validate the entire body instance using the `validate()` method.  This will check all fields, and sub-fields to ensure that all of the data is valid.  For example:

In [45]:
rqst.data.vid = 10
rqst.data.name = 'Purple'
rqst.data.validate()

True

## Accessing the Body as Dictionary Data

If you want to access the body data as dictionary structure, you can use the `as_dict()` method.  For example:

In [46]:
rqst.data.as_dict()

{u'name': 'Purple', u'vid': 10}

You can also treat the body instance like a dictionary, using methods such as `items()`, `keys()`, and `values()`.  For example:

In [47]:
rqst.data.items()

[(u'status', None),
 (u'group', None),
 (u'name', <Literal<str> Purple>),
 (u'vid', <Literal<int> 10>),
 (u'site', None),
 (u'role', None),
 (u'custom_fields', None),
 (u'tenant', None),
 (u'description', None)]

<div class="alert alert-info">
<strong>Info! &nbsp;</strong>When you execute a Request, the halutz library will automatically use the body parameter as a dictionary.  You do not need to explicity convert the data to a dictionary.
</div>

## Introspecting the Body Fields

You can introspec the body fields to see their specific schema values.  To do this, you can use the `propinfo()` method.  For example, if we wanted to know more about the `vid` field:

In [48]:
rqst.data.propinfo('vid')

{u'description': u'', 'raw_name': u'vid', u'type': u'integer'}

You can also introspect the field type.  To do this you use the `proptype()` method.  For example:

In [49]:
rqst.data.proptype('vid')

int

You can also get the complete Swagger schema specification using the special `__propinfo__` attribute.  For example:

In [50]:
rqst.data.__propinfo__

{u'custom_fields': {u'description': u'',
  'raw_name': u'custom_fields',
  u'type': u'string'},
 u'description': {u'description': u'',
  'raw_name': u'description',
  u'type': u'string'},
 u'group': {u'description': u'', 'raw_name': u'group', u'type': u'string'},
 u'name': {u'description': u'', 'raw_name': u'name', u'type': u'string'},
 u'role': {u'description': u'', 'raw_name': u'role', u'type': u'string'},
 u'site': {u'description': u'', 'raw_name': u'site', u'type': u'string'},
 u'status': {u'description': u'', 'raw_name': u'status', u'type': u'string'},
 u'tenant': {u'description': u'', 'raw_name': u'tenant', u'type': u'string'},
 u'vid': {u'description': u'', 'raw_name': u'vid', u'type': u'integer'}}

## Nested Body Schema Fields

You may encounter body schemas that have deeply nested data structures - lists of objects, objects of objects, etc.  In these cases, you can assign these fields in one of two ways.  The first way is to make the assignment using a native python structures of lists and dictionaries.  The second way is to create an instance of the sub-field type.

### When Body Field is a Dict (object)
To demonstrate this, we will load an example schema with a nested structure.

In [74]:
schema = json.load(open('nested-schema.json'))

In [75]:
print json.dumps(schema, indent=2)

{
  "x-model": "Example", 
  "type": "object", 
  "properties": {
    "port_speed": {
      "required": [
        "value", 
        "unit"
      ], 
      "type": "object", 
      "properties": {
        "value": {
          "enum": [
            0, 
            1, 
            100, 
            40, 
            10, 
            50, 
            25
          ], 
          "type": "integer", 
          "description": "scalar multiplier for the speed of the port", 
          "format": "int32"
        }, 
        "unit": {
          "enum": [
            "", 
            "M", 
            "G"
          ], 
          "type": "string", 
          "description": "Indicates if 'value' is in units of Gbps or Mbps"
        }
      }
    }
  }
}


<div class="alert alert-warning">
<strong>NOTICE! &nbsp;</strong> As part of the halutz client, there is a schema class factory called `build`.  You generally would not need to use this as a part of normal Request usage.  I'm using this here simply to do the same meta-class creation that the is done when a Request with a body parameter is created.
</div>



In [76]:
ExampleType = client.build.schema_class(schema, schema['x-model'])

In [77]:
ExampleType

abc.Example

In [78]:
example = ExampleType()

In [79]:
example.keys()

[u'port_speed']

The `port_speed` is a nested object, which we can see by introspection:

In [80]:
example.propinfo('port_speed')

{u'properties': {u'unit': {u'description': u"Indicates if 'value' is in units of Gbps or Mbps",
   u'enum': [u'', u'M', u'G'],
   'raw_name': u'unit',
   u'type': u'string'},
  u'value': {u'description': u'scalar multiplier for the speed of the port',
   u'enum': [0, 1, 100, 40, 10, 50, 25],
   u'format': u'int32',
   'raw_name': u'value',
   u'type': u'integer'}},
 'raw_name': u'port_speed',
 u'required': [u'value', u'unit'],
 u'type': abc.port_speed_<anonymous>}

Now we could assign the port_speed in one of two ways.  The first is using a native python dictionary:

In [81]:
example.port_speed = dict(unit='G', value=40)

Or we could obtain a meta-class of the port_speed type and use that.  For example:

In [82]:
PortSpeedType = example.proptype('port_speed')

In [83]:
example.port_speed = PortSpeedType(unit='G', value=40)

And now we can introspect the data as a dictionary to see that this approach is equivalent to the native python approach:

In [84]:
example.as_dict()

{u'port_speed': {u'unit': 'G', u'value': 40}}

So which approach should you use -- native python or jsonschema-typed?  The end results are equivalent.  The benefit of using the jsonschema-typed approach are twofold.  First, if you are using the halutz client interactively, you can continue to introspect and assign values as you go along.  Second, the jsonschema-typed approach ensures that your data is validated as you naviagte nested structures.  This approach is easier to debug and troubleshoot as you use the IPython shell.

### When Body Field is a List (array)

Sometimes you may be working with a nested schema that has lists of objects.  For example the following is a schema that has a toplevel field called "interfaces".  This "interfaces" is a list of objects, each object having an interface-name (if_name) and a port-speed.  The port-speed is also an object.

In [85]:
schema = json.load(open("listof-nested-schema.json"))
print json.dumps(schema, indent=2)

{
  "x-model": "ListExampleType", 
  "type": "object", 
  "properties": {
    "interfaces": {
      "items": {
        "type": "object", 
        "properties": {
          "if_name": {
            "type": "string"
          }, 
          "port_speed": {
            "required": [
              "value", 
              "unit"
            ], 
            "type": "object", 
            "properties": {
              "value": {
                "enum": [
                  0, 
                  1, 
                  100, 
                  40, 
                  10, 
                  50, 
                  25
                ], 
                "type": "integer", 
                "description": "scalar multiplier for the speed of the port", 
                "format": "int32"
              }, 
              "unit": {
                "enum": [
                  "", 
                  "M", 
                  "G"
                ], 
                "type": "string", 
                "description":

In these situations, when you make a call to `proptype` you are getting back the class for an item within the list.  For example:

In [88]:
ListExampleType = client.build.schema_class(schema, schema['x-model'])

In [89]:
item = ListExampleType()

In [90]:
item.keys()

[u'interfaces']

In [91]:
InterfaceType = item.proptype('interfaces')

In [92]:
an_interface = InterfaceType()

Now when we introspect an instance of this list item, we can see the fields: if_name and port_speed.

In [94]:
an_interface

<interfaces_<anonymous_field> if_name=None port_speed=None>

To add the interface to the `interfaces` list. You can make an assignment as the complete list, for example:

In [98]:
an_interface.if_name = 'eth1'
an_interface.port_speed = dict(unit='G', value=40)

item.interfaces = [an_interface]

print json.dumps(item.as_dict(), indent=2)

{
  "interfaces": [
    {
      "if_name": "eth1", 
      "port_speed": {
        "value": 40, 
        "unit": "G"
      }
    }
  ]
}


Alternatively, you could initialize the `interfaces` list to an empty list, and then append items as you go along.  For example:

In [105]:
item.interfaces = []

In [106]:
item.interfaces.append(an_interface)

# add another interface using the InterfaceType, just to show this example

item.interfaces.append(InterfaceType(if_name='eth2', port_speed=dict(unit='G', value=10)))

In [107]:
print json.dumps(item.as_dict(), indent=2)

{
  "interfaces": [
    {
      "if_name": "eth1", 
      "port_speed": {
        "value": 40, 
        "unit": "G"
      }
    }, 
    {
      "if_name": "eth2", 
      "port_speed": {
        "value": 10, 
        "unit": "G"
      }
    }
  ]
}


## Executing a Request with Body Parameters

Again is should be noted that when you are using a Request that has a body parameter, you **DO NOT** need to pass that parameter on the Request call.  For example, if we wanted to create a new VLAN.

In [86]:
rqst = client.request.ipam.ipam_vlans_create

rqst.data.vid = 1001
rqst.data.name = "Marketing"
rqst.data.validate()

# execute the request; it knows to use the body parameter so you don't need to 
# pass it explicity

resp, ok = rqst()

# Next Topics

  - <a href="Request-Responses.ipynb">Learn to work with Request responses</a>